# &#128214; Lab 8: Static Taint Analysis

## &#127919; Objective

Explore static taint analysis, understand the concepts of sources, sinks, and taint propagation, and apply these concepts to identify potential security leaks in a program.

## &#128214; Background

Static taint analysis is a technique used to track the flow of sensitive information through a program. It involves marking certain inputs as "tainted" and then analyzing the code to see where this tainted data goes. The goal is to ensure tainted data does not reach leaking points, known as "sinks".

### Key Concepts

- **Source:** Entry point where sensitive data enters the program (e.g., user input or sensitive data).
- **Sink:** A point in the program where tainted data flows (usually out of the software space, e.g., through the network).
- **Taint Propagation:** Process of tracing the flow of tainted data through the program.

In general, taint analysis is the process of following certain data from program points to other program points.
In practice it is used to detect vulnerabilities (e.g., SQL injection) or sensitive data leaks.

## &#10145; Tasks

1. **Understanding Taint Analysis:**
   - Familiarize yourself with the concept of taint analysis and how it is used in data flow analysis.

2. **Implementing Taint Analysis:**
   - Use the `ControlFlowGraph` class to support the computation of the taint analysis.
   - Implement the `get_gen` and `get_kill` functions from the previous abstract `DataFlowAnalysis` class.
   - Verify your implementation with several examples.

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation
- `abc`: a module to implement abstract methods in Python

In [ ]:
import ast
from abc import ABC, abstractmethod

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
def get_sensitive_data():
    return "sensitive"

def leak_sensitive_data(data):
    print(data)

a = get_sensitive_data()
b = a
a = 1
leak_sensitive_data(b)
"""

### Utility Control Flow Graph class

&#128161; The following cell contains a utility class to build a Control Glow Graph. 
You have to use this class to build the control flow graph.

In [ ]:
class ControlFlowGraph:
    """
    A class representing a Control Flow Graph (CFG).

    Attributes:
    nodes: A list where each element is a statement.
    edges: A list of tuples representing edges between nodes, where each tuple contains a pair of nodes.

    Methods:
    add_node:
        Adds a new node with the given statement to the graph, returning the new node.
    add_edge:
        Adds an edge between the specified node indices to the graph.
    visualize:
        Prints a visualization of the graph to the console.
    to_dot:
        Returns a DOT-format string representing the graph.
    """

    def __init__(self):
        """
        Initializes a new instance of the ControlFlowGraph class, with empty nodes and edges.
        """
        self.nodes = []
        self.edges = []

    def add_node(self, statement):
        """
        Adds a new node with the given statement to the graph.

        Parameters:
        statement: The statement associated with the new node.

        Returns:
        The node
        """
        self.nodes.append(statement)
        return statement

    def add_edge(self, node1, node2):
        """
        Adds an edge between the specified nodes to the graph.

        Parameters:
        node1: The source node.
        node2: The target node.
        """
        if (node1, node2) not in self.edges:
            self.edges.append((node1, node2))

    def visualize(self):
        """
        Prints a visualization of the graph to the console.
        Each edge is printed as a line in the format 'source -> target'.
        """
        for edge in self.edges:
            source, target = edge
            print(f'{source} -> {target}')

    def to_dot(self):
        """
        Returns a DOT-format string representing the graph (for vizualization purposes).

        Returns:
        str: A string in DOT format.
        """
        dot_lines = ['digraph cfg {']
        stmt_to_index = {}
        for index, statement in enumerate(self.nodes):
            stmt_to_index[statement] = index
            dot_lines.append(f'    node{index} [label="{statement}"];')
        for edge in self.edges:
            source, target = edge
            dot_lines.append(f'    node{stmt_to_index[source]} -> node{stmt_to_index[target]};')
        dot_lines.append('}')
        return '\n'.join(dot_lines)

### Function to create the CFG

&#128161; In the following cell, you will use the `build_cfg` function of the previous lab.

In [ ]:
def build_cfg(node, cfg, parent=None):
    """
    Recursively builds a Control Flow Graph (CFG) from an Abstract Syntax Tree (AST) node, considering various types of statements including assignment, expression, conditional, loop, and module level statements.

    Parameters:
    node: The current node in the AST.
    cfg: An instance of ControlFlowGraph to which nodes and edges will be added.
    parent: A list of parent nodes from which edges to the current node will be drawn. 
    
    Returns:
    list: A list of current nodes which will act as parents for the next level of recursion.
    """

### Build the CFG

&#128161; In the following cell, you will build the CFG as in previous labs

&#10067; Before implementing the Taint Analysis

Explain the following:
- What is a **data flow fact** in the case of taint analysis?
- How would you store the data flow facts?
- What do the `in` and `out` sets represent with the taint analysis?
- How would you **generate** a data flow fact, in other word, what is the GEN function?
- How would you **kill** a data flow fact, in other word, what is the KILL function?
- How would you propagate the *taint*?

### Implement the Taint Analysis

&#128161; In the following cell, you will implement the *taint analysis* using a class called `TaintAnalysis` that extends the  `DataFlowAnalysis` used in Lab 4.

The `TaintAnalysis` class should take two additional sets in parameter:
1. a `sources` set that represent the set of sources to consider (in our case just the name of functions), i.e., the ones that will generate data flow facts
2. a `sinks` set that represent the set of sinks to consider (in our case just the name of functions), i.e., the ones that will be used to check if there is a leak in the program.

In [ ]:
class DataFlowAnalysis(ABC):
    """
    Abstract Base Class for data flow analysis on a Control Flow Graph (CFG).

    Attributes:
        cfg: Control Flow Graph on which to perform the analysis.
        in_sets: Dictionary to store 'in' sets for each node.
        out_sets: Dictionary to store 'out' sets for each node.
    """

    def __init__(self, cfg):
        """
        Initialize the DataFlowAnalysis class.

        Parameters:
            cfg: Control Flow Graph on which to perform the analysis.
        """

        self.cfg = cfg
        self.in_sets = {node: set() for node in cfg.nodes}
        self.out_sets = {node: set() for node in cfg.nodes}

    def analyze(self):
        """
        Core algorithm for data flow analysis, common to all types of analyses.
        
        Updates the in_sets and out_sets attributes based on the implemented get_gen and get_kill methods.
        """

        first_id = 1
        worklist = [self.cfg.nodes[first_id]]
        visited = set()

        while worklist:
            node = worklist.pop(0)
            visited.add(node)

            # In[node] = Union of Out[predecessors]
            in_set = set().union(
                *[self.out_sets[pred] for pred, succ in self.cfg.edges if succ == node]
            )
            self.in_sets[node] = in_set

            # Out[node] = gen U (In[node] - kill)
            gen = self.get_gen(node)
            kill = self.get_kill(gen, in_set, node)
            out_set = gen.union(in_set - kill)
            self.out_sets[node] = out_set

            successors = {succ for pred, succ in self.cfg.edges if pred == node}
            worklist.extend(succ for succ in successors if succ not in visited)

    @abstractmethod
    def get_gen(self, node):
        """
        Abstract method to get the 'gen' set.
        
        Must be implemented by subclass.

        Returns:
            The 'gen' set.
        """
        pass

    @abstractmethod
    def get_kill(self, gen, in_set, node):
        """
        Abstract method to get the 'kill' set.
        
        Must be implemented by subclass.

        Parameters:
            gen: The 'gen' set.
            in_set: The 'in' set for a node.
            node: the node under analysis

        Returns:
            The 'kill' set.
        """
        pass

In [ ]:
class TaintAnalysis(DataFlowAnalysis, ABC):
    
    def __init__(self, cfg, sources, sinks):
        super().__init__(cfg)
        self.sources = sources
        self.sinks = sinks

&#128161; Before testing your code, provide the in and out sets for all statements of the code in the following cell: 

#### Test your code

&#128161; In the following cell, you will instantiate a `TaintAnalysis` object with the CFG as a parameter and a list of sources and sinks and trigger the taint analysis that should populate the `in` and `out` sets for each node of the CFG.
Print the out set of each node.

In [ ]:
sources = set()
sources.add("get_sensitive_data")
sinks = set()
sinks.add("leak_sensitive_data")
ta = TaintAnalysis(cfg, sources, sinks)

&#128161; Perfect, you can now follow taints across a program, but you did not yet answer the question: is there a leak in the program?
Obviously there is one, but your program does not yet answer this question, how would you implement it?

&#128161; In the following cell, implement a small algorithm to check if there is a leak in the program.

## Let us make the code more complicated

Python code

&#128161; The new code on which you will apply your analyses

In [ ]:
code = """

def get_sensitive_data():
    return "sensitive"

def leak_sensitive_data(data):
    print(data)

condition = input()

a = get_sensitive_data()

if condition:
    b = 1
else:
    b = a

leak_sensitive_data(b)
"""

&#128161; In the following cell, you will parse the code, build the cfg, run the taint analysis on the new code, and check if there is a leak.

&#10067; Is this output expected?
Should there be a leak?
Should the static analysis return a leak here?

&#128161; So far, our list of sources was simple: only one method.

But what if there are more function? How would you know what method was the source of a leak?

For instance, if the list of sources contain the following:
`sources = ["get_username", "get_password"]` and we want to know whether the username or the password has been leaked, how to do that?

Change the implementation of your taint analysis and the leak check to output this information.

In [ ]:
class TaintAnalysis(DataFlowAnalysis, ABC):
    
    def __init__(self, cfg, sources, sinks):
        super().__init__(cfg)
        self.sources = sources
        self.sinks = sinks

Python code

&#128161; The new code on which you will apply your analyses

In [ ]:
code = """

condition = input()

a = get_password()

if condition:
    b = 1
else:
    b = a

leak_sensitive_data(b)
"""

&#128161; Now run the analysis and check if there is a leak.

&#10067; How would you check for an SQL injection with this technique? 

&#128161; Now let us redo our taint analysis on the following piece of code:

In [ ]:
code = """

def bridge(data):
    leak_sensitive_data(data)

a = get_password()
bridge(a)
"""

&#10067; Does your analysis return a leak?

We will see why in the next lab.